In [1]:
using Pkg; Pkg.activate("./binder/")

Activating environment at `~/work/cfgrib-julia-notebooks/binder/Project.toml`


# Intro

This notebook goes through the following climate/grib-related packages:

 - [ClimateTools.jl](https://github.com/Balinus/ClimateTools.jl)
    - Defines CF-compliant ClimGrid type
 - [GRIB.jl](https://github.com/weech/GRIB.jl)
    - Uses ecCodes library, loads GRIB files in series message-by-message
    - Has filtering functionality to only load in specific keys
 - [GDAL.jl](https://github.com/JuliaGeo/GDAL.jl)
 - [ArchGDAL.jl](https://github.com/yeesian/ArchGDAL.jl)
 - [ICOADSDict](https://github.com/JuliaAtmosOcean/ICOADSDict.jl)

## Conclusion

[GRIB.jl](https://github.com/weech/GRIB.jl) is a package which calls the ecCodes to read GRIB files.

As per the readme:

> A GribFile functions similarly to a Julia IOStream, except that instead of working as a stream of bytes, GribFile works as a stream of messages.

It may be possible to use `GRIB.jl` to load in the data, then place it into an `AxisArray` and add in the metadata as it streams in. This situation where the types of data may only become clear later as the data streams in would be a place where `SimpleTraits.jl` could help ensure correct typing.

## ClimateTools.jl

https://github.com/Balinus/ClimateTools.jl

> ClimateTools.jl is a collection of commonly-used tools in Climate science. Basics of climate field analysis are covered, with some forays into exploratory techniques associated with climate scenarios design. The package is aimed to ease the typical steps of analysis climate models outputs and gridded datasets (support for weather stations is a work-in-progress).
> 
> Climate indices and bias correction functions are coded to leverage the use of multiple threads. To gain maximum performance, use (bash shell Linux/MacOSX) export JULIA_NUM_THREADS=n, where n is the number of threads. To get an idea of the number of threads you can use type (in Julia) Sys.THREADS. This is especially useful for bias correction.

### Features:

> Climate scenarios creation
> Extraction and visualization of CF-compliant netCDF datasets
> Custom user-provided polygons and start and end date for localized studies
> Climate indices from The joint CCl/CLIVAR/JCOMM Expert Team (ET) on Climate Change Detection and Indices (ETCCDI) as well as custom climate indices
> Regridding of a datasets onto another grid
> Post-processing of climate timeseries using Quantile-Quantile mapping method (cf. Themeßl et al. 2012, Piani et al. 2010)
> Exportation of results to a CF-compliant netCDF file
> Support for typical climate models calendars: 360day, 365day, Standard, Prolectip Gregorian through NCDatasets.jl.
> Support for physical units through the Unitful.jl package.


Provides a `ClimGrid` CF-compliant type:

```julia
struct ClimGrid
  data::AxisArray # labeled axis
  longrid::AbstractArray{N,2} where N # the longitude grid
  latgrid::AbstractArray{N,2} where N # the latitude grid
  msk::Array{N, 2} where N
  grid_mapping::Dict # bindings of native grid
  dimension_dict::Dict
  model::String
  frequency::String
  experiment::String
  run::String
  project::String # CORDEX, CMIP5, etc.
  institute::String
  filename::String
  dataunits::String
  latunits::String # of the coordinate variable
  lonunits::String # of the coordinate variable
  variable::String # Type of variable (i.e. can be the same as "var", but it is changed when calculating indices)
  typeofvar::String # Variable type (e.g. tasmax, tasmin, pr)
  typeofcal::String # Calendar type
  timeattrib::Dict # Time attributes
  varattribs::Dict # Variable attributes
  globalattribs::Dict # Global attributes
end
```

Provides interpolation functions **by wrapping scipy griddata functions**

Provides bias correction as per *Themeßl et al. 2012*, *Grenier et al. 2015*

Integrated visualisation tools: https://balinus.github.io/ClimateTools.jl/stable/maps/

## GRIB.jl

https://github.com/weech/GRIB.jl

> The Gridded Binary (GRIB) format is a format commonly used in meteorology. A GRIB file is a collection of independent records that store 2D data. **This package is an interface to the ECMWF ecCodes library**. In ecCodes, each GRIB file is composed of a series of messages, and a message is an object with keys and values. Each message has many keys. Some are actually stored in the data, while others are computed by ecCodes on access. Some commonly used keys include:

In [2]:
using GRIB

In [3]:
f = GribFile("./data/era5-levels-members.grib")

GribFile ./data/era5-levels-members.grib at position 0 in mode r

> A GribFile functions similarly to a Julia IOStream, except that instead of working as a stream of bytes, GribFile works as a stream of messages.

In [4]:
methodswith(GribFile)

[1] eltype(f::GribFile) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/gribfile.jl:67
[2] iterate(f::GribFile) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/gribfile.jl:59
[3] iterate(f::GribFile, state) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/gribfile.jl:59
[4] position(f::GribFile) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/gribfile.jl:108
[5] read(f::GribFile) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/gribfile.jl:101
[6] read(f::GribFile, nm::Integer) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/gribfile.jl:92
[7] seek(f::GribFile, n::Integer) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/gribfile.jl:116
[8] seekstart(f::GribFile) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/gribfile.jl:70
[9] show(io::IO, mime::MIME{Symbol("text/plain")}, f::GribFile) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/gribfile.jl:165
[10] skip(f::GribFile, offset::Integer) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/gribfile.jl:133
[11] destroy(f::GribFile) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/gribfile.jl:150

In [5]:
m = Message(f)

date     gridType       stepRange typeOfLevel       level shortName name
20170101 regular_ll     0         isobaricInhPa     500   z         Geopotential

In [6]:
methodswith(Message)

[1] deepcopy(handle::Message) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:326
[2] getindex(handle::Message, key::AbstractString) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:184
[3] haskey(msg::Message, key::AbstractString) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:113
[4] iterate(iter::Message) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/messageiters.jl:53
[5] iterate(iter::Message, state) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/messageiters.jl:53
[6] keys(handle::Message) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/messageiters.jl:75
[7] setindex!(handle::Message, value::Integer, key::AbstractString) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:261
[8] setindex!(handle::Message, value::AbstractFloat, key::AbstractString) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:267
[9] setindex!(handle::Message, value::AbstractString, key::AbstractString) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:273
[10] setindex!(handle::Message, value::Array{UInt8,1}, key::AbstractString) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:281
[11] setindex!(handle::Message, value::Array{Float64,N} where N, key::AbstractString) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:289
[12] setindex!(handle::Message, value::Array{Int64,N} where N, key::AbstractString) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:297
[13] setindex!(handle::Message, value::Array{String,N} where N, key::AbstractString) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:305
[14] show(io::IO, mime::MIME{Symbol("text/plain")}, message::Message) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:373
[15] show(io::IO, message::Message) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:383
[16] values(handle::Message) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/messageiters.jl:93
[17] clone(handle::Message) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:314
[18] data(handle::Message) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:343
[19] destroy(handle::Message) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:364
[20] eachpoint(handle::Message) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/messageiters.jl:126
[21] find(near::Nearest, handle::Message, inlon, inlat) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/nearest.jl:62
[22] findmultiple(handle::Message, inlons::Array{Float64,1}, inlats::Array{Float64,1}) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/nearest.jl:96
[23] getbytes(message::Message) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:90
[24] maskedvalues(msg::Message) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:140
[25] missingvalue(msg::Message) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:127
[26] writemessage(handle::Message, filename::AbstractString) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/message.jl:107

In [7]:
d = data(m)

([0.0 0.0 … 0.0 0.0; 3.0 3.0 … 3.0 3.0; … ; 354.0 354.0 … 354.0 354.0; 357.0 357.0 … 357.0 357.0], [90.0 87.0 … -87.0 -90.0; 90.0 87.0 … -87.0 -90.0; … ; 90.0 87.0 … -87.0 -90.0; 90.0 87.0 … -87.0 -90.0], [51169.703125 51202.453125 … 50951.703125 50866.453125; 51169.703125 51213.703125 … 50965.203125 50866.453125; … ; 51169.703125 51178.453125 … 50924.453125 50866.453125; 51169.703125 51190.953125 … 50938.203125 50866.453125])

In [8]:
[print("$(size(v)), ") for v in d];

(120, 61), (120, 61), (120, 61), 

In [9]:
read(f, 5)[1]

date     gridType       stepRange typeOfLevel       level shortName name
20170101 regular_ll     0         isobaricInhPa     500   z         Geopotential

In [10]:
destroy(f)

In [11]:
index = Index("./data/era5-levels-members.grib", "shortName", "typeOfLevel", "level")

select!(index, "shortName", "z")
select!(index, "typeOfLevel", "isobaricInhPa")
select!(index, "level", 500)

In [12]:
methodswith(Index)

[1] eltype(f::Index) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/index.jl:120
[2] iterate(f::Index) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/index.jl:112
[3] iterate(f::Index, state) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/index.jl:112
[4] show(io::IO, mime::MIME{Symbol("text/plain")}, index::Index) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/index.jl:129
[5] addfile!(index::Index, filename::AbstractString) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/index.jl:55
[6] destroy(idx::Index) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/index.jl:124
[7] keycount(index::Index, key::AbstractString) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/index.jl:65
[8] select!(index::Index, key::AbstractString, value::AbstractString) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/index.jl:93
[9] select!(index::Index, key::AbstractString, value::AbstractFloat) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/index.jl:99
[10] select!(index::Index, key::AbstractString, value::Integer) in GRIB at /home/roscar/.julia/packages/GRIB/MoOdx/src/index.jl:105

Quite neat, may be able to build on top of this package instead of creating an interface to ecCodes from scratch.

## GDAL.jl

https://github.com/JuliaGeo/GDAL.jl

> Julia wrapper for GDAL - Geospatial Data Abstraction Library. This package is a binding to the C API of GDAL/OGR. It provides only a C style usage, where resources must be closed manually, and datasets are pointers. Note: this also means that before being able to use any drivers (for input/output of different formats), they must be registered first using GDAL.gdalallregister().

## ArchGDAL.jl

https://github.com/yeesian/ArchGDAL.jl

> GDAL is a translator library for raster and vector geospatial data formats that is released under an X/MIT license by the Open Source Geospatial Foundation. As a library, it presents an abstract data model to drivers for various raster and vector formats.
> 
> This package aims to be a complete solution for working with GDAL in Julia, similar in scope to the SWIG bindings for Python. It builds on top of GDAL.jl, and provides a high level API for GDAL, espousing the following principles.

Adds some more user-friendly features on top of GDAL.jl, but overall is still quite low level and basically just binds the C API.

## ICOADSDictionary.jl

https://github.com/JuliaAtmosOcean/ICOADSDict.jl

> The International Comprehensive Ocean-Atmosphere Data Set (ICOADS) is a compilation of the world’s in situ surface marine observations and represents a culmination of efforts to digitize, assemble, and reconcile information collected over the years by many countries. This package returns a dictionary of symbols and alphanumeric values from an ICOADS IMMA file.

Not as related as I expected at first